In [1]:
from qiskit.converters import circuit_to_dag, dag_to_circuit
from helper_fun import load_IBMQ
from qiskit.transpiler.passes import NoiseAdaptiveLayout
from qiskit.providers.aer import noise
import datetime as dt
from qcg.generators import gen_supremacy, gen_hwea
from qiskit.circuit.classicalregister import ClassicalRegister
from qiskit import QuantumCircuit
from qiskit.compiler import transpile

provider = load_IBMQ()

circ = gen_supremacy(2,2,4,order='75601234')
c = ClassicalRegister(len(circ.qubits), 'c')
meas = QuantumCircuit(circ.qregs[0], c)
meas.barrier(circ.qubits)
meas.measure(circ.qubits,c)
qc = circ+meas

device = provider.get_backend('ibmq_16_melbourne')
properties = device.properties(dt.datetime(day=16, month=10, year=2019, hour=20))
coupling_map = device.configuration().coupling_map
noise_model = noise.device.basic_device_noise_model(properties)
basis_gates = noise_model.basis_gates
dag = circuit_to_dag(circ)
noise_mapper = NoiseAdaptiveLayout(properties)
noise_mapper.run(dag)
initial_layout = noise_mapper.property_set['layout']

new_circuit = transpile(qc, backend=device, basis_gates=basis_gates,coupling_map=coupling_map,backend_properties=properties,initial_layout=initial_layout)

print(new_circuit)

┌─────────────────────┐           ┌───┐                    ░ »
      (q00) q0|0>┤ U3(pi/4,-pi/2,pi/2) ├───────────┤ X ├────────────────────░─»
                 └─┬──────────────────┬┘           └─┬─┘       ┌──────────┐ ░ »
      (q01) q1|0>──┤ U3(pi/2,pi/4,pi) ├───■──────────■─────────┤ U2(0,pi) ├─░─»
                 ┌─┴──────────────────┴┐┌─┴─┐┌────────────────┐└──────────┘ ░ »
      (q03) q2|0>┤ U3(pi/4,-pi/2,pi/2) ├┤ X ├┤ U3(pi/2,pi,pi) ├─────────────░─»
                 ├─────────────────────┤└───┘└────────────────┘             ░ »
      (q02) q3|0>┤ U3(pi/4,-pi/2,pi/2) ├────────────────────────────────────░─»
                 └─────────────────────┘                                    ░ »
 (ancilla0) q4|0>─────────────────────────────────────────────────────────────»
                                                                              »
 (ancilla1) q5|0>─────────────────────────────────────────────────────────────»
                                                         

In [2]:
print(initial_layout)

Layout({
0: Qubit(QuantumRegister(4, 'q0'), 0),
1: Qubit(QuantumRegister(4, 'q0'), 1),
3: Qubit(QuantumRegister(4, 'q0'), 2),
2: Qubit(QuantumRegister(4, 'q0'), 3),
4: Qubit(QuantumRegister(10, 'ancilla'), 0),
5: Qubit(QuantumRegister(10, 'ancilla'), 1),
6: Qubit(QuantumRegister(10, 'ancilla'), 2),
7: Qubit(QuantumRegister(10, 'ancilla'), 3),
8: Qubit(QuantumRegister(10, 'ancilla'), 4),
9: Qubit(QuantumRegister(10, 'ancilla'), 5),
10: Qubit(QuantumRegister(10, 'ancilla'), 6),
11: Qubit(QuantumRegister(10, 'ancilla'), 7),
12: Qubit(QuantumRegister(10, 'ancilla'), 8),
13: Qubit(QuantumRegister(10, 'ancilla'), 9)
})
